<a href="https://colab.research.google.com/github/123-code/Deep-Learning-From-Scratch/blob/main/Micrograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install micrograd


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
from micrograd.engine import Value
import math

class Value:
  def __init__(self,data,_children=(),_op="",label=""):
    self.data = data
    self.grad = 0
    self._backward = lambda: None
    self.prev = set(_children)
    self._op = _op
    self.label = label
  
  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self,other):
    result = Value(self.data + other.data,(self,other),"+")

    def _backward():
      self.grad = 1.0 * result.grad
      other.grad = 1.0 * result.grad
    result._backward = _backward

    return result

  def __mul__(self,other):
    result = Value(self.data * other.data,(self,other),"*")

    def _backward():
      self.grad = other.data * result.grad
      other.grad = self.data * result.grad

    result._backward = _backward

      
      
    return result

  def tanh(self):
    x = self.data
    t = (math.exp(2*x)-1)/(math.exp(2*x)+12)
    out = Value(t,(self,),'tanh')

    def _backward():
      self.grad = (1- t**2) * out.grad
    out._backward = _backward
    return out

a = Value(2.0,label="a")
b =  Value(-3.0,label="b")
c = Value(10.0,label="c")
e = a*b; e.label="e"
#a + b
d = e + c; d.label="d"
f = Value(-2.0,label="f")
l = d+f
l
# variable grad should store the gradient of the loss with respect to each value. 



Value(data=2.0)

In [31]:
L = d*f
f.grad = 4.0
d.grad = -2


In [32]:
l.grad = 1

In [33]:
#Dl/dc
c.grad = -2.0
e.grad = -2.0

In [34]:
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
f.data += 0.01 * f.grad

e = a*b
d = e+c
l = d*f
print(l.data)

-7.800800000000001


Example of gradient check: Derivative of the loss against all the previous multiplication results.

In [35]:
def lol():
  h=0.001
  a = Value(2.0,label="a")
  b =  Value(-3.0,label="b")
  c = Value(10.0,label="c")
  e = a*b; e.label="e"
#a + b
  d = e + c; d.label="d"
  f = Value(-2.0,label="f")
  l = d*f
  l1 = l.data


  
  a = Value(2.0 ,label="a")
  b =  Value(-3.0,label="b")
  c = Value(10.0,label="c")
  e = a*b; e.label="e"
  e.data += h
#a + b
  d = e + c; d.label="d"
  f = Value(-2.0 ,label="f")
  l = d*f
  l2 = l.data
  print((l2-l1)/h)

lol()


-2.000000000000668


Neuron basic code

In [44]:
import numpy as np

In [43]:
#inputs x1,x2
x1 = Value(2.0,label='x1')
x2=Value(0.0,label='x2')
#weights w1,w2
w1=Value(-3.0,label='w1')
w2=Value(1.0,label='w2')

# neuron bias
b = Value(6.7,label='b')

x1w1 = np.dot(x1,w1)
x2w2 = np.dot(x2,w2)
wsum = x1w1 * x2w2 + b
o = wsum.tanh()
print(o)
print(wsum)

#apply activation function




Value(data=0.9999803034846588)
Value(data=6.7)


In [45]:
o.grad = 1.0

In [46]:
o._backward()

In [47]:
wsum._backward()

In [23]:
o.grad = 1.0
1-o.data**2

3.939264272967424e-05